In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import joblib
import os
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from pathlib import Path

In [3]:
# Function to preprocess data
def preprocess_data(df):
    df.columns = df.columns.str.replace(' ', '_', regex=False)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month
    df['year'] = df['datetime'].dt.year
    df['hour'] = df['datetime'].dt.hour
    df['weekday'] = df['datetime'].dt.weekday
    return df

In [4]:
# Function to add lag features
def add_lag_features(df, target_col, lags):
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)
    df.dropna(inplace=True)
    return df

In [5]:
# Function to calculate metrics
def calculate_metrics(actual, forecast):
    mse = mean_squared_error(actual, forecast)
    mae = mean_absolute_error(actual, forecast)
    r2 = r2_score(actual, forecast)
    rmse = np.sqrt(mse)
    bias = np.mean(forecast) - np.mean(actual)
    return {
        'Mean Squared Error': mse,
        'Mean Absolute Error': mae,
        'R^2 Score': r2,
        'Root Mean Squared Error': rmse,
        'Bias': bias
    }

In [6]:
# Function to train and forecast with LightGBM
def lightgbm_forecast(df, target_col, train_end_date, is_log_transformed=False, n_splits=5):
    train_data = df[df['datetime'] <= train_end_date]
    test_data = df[df['datetime'] > train_end_date]

    if train_data.empty or test_data.empty:
        raise ValueError("Train or test data is empty. Please check the data split.")

    exclude_cols = ['datetime', 'index']

    base_pollutant = target_col.replace('_log', '').split('_')[0]
    station_suffix = '_'.join(target_col.replace('_log', '').split('_')[1:])

    base_features = [
        'direct_radiation_(W/m²)',
        f'RH_{station_suffix}',
        f'TMP_{station_suffix}',
        f'WDR_{station_suffix}',
        f'WSP_{station_suffix}',
        'is_festival',
        'is_weekend',
        f'{target_col}_lag_1',
        f'{target_col}_lag_2',
        f'{target_col}_lag_3',
        f'{target_col}_lag_24',
        f'{target_col}_lag_48',
        f'{target_col}_lag_72',
        'day',
        'month',
        'year',
        'hour',
        'weekday'
    ]

    selected_features = [feat for feat in base_features if feat in train_data.columns]

    X_train = train_data[selected_features]
    X_test = test_data[selected_features]

    y_train = train_data[target_col]
    y_test = test_data[target_col]

    print("Extracted Features:")
    for feature in X_train.columns:
        print(feature)

    cv = TimeSeriesSplit(n_splits=n_splits)

    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': 50,
        'learning_rate': 0.05,
        'feature_fraction': 0.8
    }

    # Cross-validation for evaluation only
    for train_idx, val_idx in cv.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        train_data_lgb = lgb.Dataset(X_train_fold, label=y_train_fold)
        val_data_lgb = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data_lgb)

        lgb.train(
            params,
            train_data_lgb,
            num_boost_round=200,
            valid_sets=[val_data_lgb],
            callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
        )

    # Final model training on full training data
    final_train_data = lgb.Dataset(X_train, label=y_train)
    final_model = lgb.train(
        params,
        final_train_data,
        num_boost_round=200
    )

    forecasts = final_model.predict(X_test)

    if is_log_transformed:
        forecasts = np.expm1(forecasts)
        y_test = np.expm1(y_test)

    metrics = calculate_metrics(y_test, forecasts)

    return forecasts, pd.Series(y_test.values, index=test_data['datetime']), metrics, final_model

In [7]:
# Function to apply log transformation
def apply_log_transform(df, target_col):
    df[f'{target_col}_log'] = np.log1p(df[target_col])
    return df

In [8]:
# Function to plot results
def plot_results(actual, forecast, target_col, train_end_date='2022-12-31'):
    if isinstance(forecast, np.ndarray):
        forecast = pd.Series(forecast, index=actual.index)
    
    test_period = actual.index > pd.Timestamp(train_end_date)
    
    plt.figure(figsize=(12, 6))
    plt.plot(actual.index, actual, label='Actual (Observed)', color='blue', linewidth=1.5)
    plt.plot(actual.index[test_period], forecast[test_period], label='Forecast (Predicted)', linestyle='--', color='orange', linewidth=1.5)
    plt.axvline(x=pd.Timestamp(train_end_date), color='red', linestyle='--', label='Train/Test Split')
    plt.title(f"{target_col} Forecast vs Actual", fontsize=16)
    plt.xlabel("Date", fontsize=12)
    plt.ylabel(f'{target_col} Values', fontsize=12)
    plt.grid(alpha=0.3, linestyle='--')
    plt.legend(fontsize=10)
    plt.tight_layout()
    plt.show()

In [9]:
# Function to save the trained model
def save_model(model, target_col):
    model_filename = f"{target_col}_model_.pkl"
    joblib.dump(model, model_filename)
    print(f"Model saved as: {model_filename}")

In [10]:
# Main workflow with and without log transformation
def main_workflow_with_analysis(file_path, target_cols, train_end_date):
    data = preprocess_data(pd.read_excel(file_path))
    
    results_no_log = {}
    results_with_log = {}
    
    for target_col in target_cols:
        print(f"Processing {target_col}...")
        
        lags = [1, 2, 3, 24, 48, 72]
        data_lagged = add_lag_features(data.copy(), target_col, lags)
        
        forecast_no_log, actual_no_log, metrics_no_log, _ = lightgbm_forecast(data_lagged, target_col, train_end_date)
        results_no_log[target_col] = {'metrics': metrics_no_log}
        plot_results(actual_no_log, forecast_no_log, target_col, train_end_date)
        
        # Apply log transform
        data_log = apply_log_transform(data.copy(), target_col)
        data_log_lagged = add_lag_features(data_log, f'{target_col}_log', lags)
        forecast_log, actual_log, metrics_log, model = lightgbm_forecast(data_log_lagged, f'{target_col}_log', train_end_date, is_log_transformed=True)
        results_with_log[target_col] = {'metrics': metrics_log}
        plot_results(actual_log, forecast_log, f"{target_col} (Log Transformed)", train_end_date)

        # Save the log-transformed model
        save_model(model, target_col)

    return results_no_log, results_with_log

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/MER_imputed_merged_data.xlsx"

pollutants = ['PM25_MER', 'PM10_MER', 'SO2_MER', 'O3_MER', 'NO2_MER', 'CO_MER']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[MER STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[MER STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/AJM_imputed_merged_data.xlsx"

pollutants = ['PM25_AJM', 'PM10_AJM', 'SO2_AJM', 'O3_AJM', 'NO2_AJM', 'CO_AJM']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[AJM STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[AJM STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/BJU_imputed_merged_data.xlsx"

pollutants = ['PM25_BJU', 'PM10_BJU', 'SO2_BJU', 'O3_BJU', 'NO2_BJU', 'CO_BJU']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[BJU STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[BJU STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/HGM_imputed_merged_data.xlsx"

pollutants = ['PM25_HGM', 'PM10_HGM', 'SO2_HGM', 'O3_HGM', 'NO2_HGM', 'CO_HGM']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[HGM STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[HGM STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/MGH_imputed_merged_data.xlsx"

pollutants = ['PM25_MGH', 'PM10_MGH', 'SO2_MGH', 'O3_MGH', 'NO2_MGH', 'CO_MGH']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[MGH STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[MGH STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/MPA_imputed_merged_data.xlsx"

pollutants = ['PM25_MPA', 'PM10_MPA', 'SO2_MPA', 'O3_MPA', 'NO2_MPA', 'CO_MPA']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[MPA STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[MPA STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/PED_imputed_merged_data.xlsx"

pollutants = ['PM25_PED', 'PM10_PED', 'SO2_PED', 'O3_PED', 'NO2_PED', 'CO_PED']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[PED STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[PED STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/SAG_imputed_merged_data.xlsx"

pollutants = ['PM25_SAG', 'PM10_SAG', 'SO2_SAG', 'O3_SAG', 'NO2_SAG', 'CO_SAG']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[SAG STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[SAG STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/SFE_imputed_merged_data.xlsx"

pollutants = ['PM25_SFE', 'PM10_SFE', 'SO2_SFE', 'O3_SFE', 'NO2_SFE', 'CO_SFE']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[SFE STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[SFE STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/TLA_imputed_merged_data.xlsx"

pollutants = ['PM25_TLA', 'PM10_TLA', 'SO2_TLA', 'O3_TLA', 'NO2_TLA', 'CO_TLA']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[TLA STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[TLA STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/UIZ_imputed_merged_data.xlsx"

pollutants = ['PM25_UIZ', 'PM10_UIZ', 'SO2_UIZ', 'O3_UIZ', 'NO2_UIZ', 'CO_UIZ']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[UIZ STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[UIZ STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
# Running the workflow with and without log transform

# Path to current data
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
current_data = f"{BASE_DIR}/2. Data Collection/XAL_imputed_merged_data.xlsx"

pollutants = ['PM25_XAL', 'PM10_XAL', 'SO2_XAL', 'O3_XAL', 'NO2_XAL', 'CO_XAL']
train_end_date = '2022-12-31'
results_no_log, results_with_log = main_workflow_with_analysis(
    file_path=current_data,
    target_cols=pollutants,
    train_end_date=train_end_date
)

In [ ]:
print("\n-[XAL STATION] Results Without Log Transform-")
for pollutant, result in results_no_log.items():
    print(f"{pollutant}: {result['metrics']}")

In [ ]:
print("\n-[XAL STATION] Results With Log Transform-")
for pollutant, result in results_with_log.items():
    print(f"{pollutant}: {result['metrics']}")